In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 폰트 깨짐 방지
import os
import numpy as np

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_rows", None)  # 행 전부 보기
pd.set_option("display.max_columns", None)  # 열 전부 보기
pd.set_option("display.width", 1000)  # 가로 폭 조절

In [2]:
file_path = os.path.join("../../dataset", "phishing_data.csv")
df = pd.read_csv(file_path)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44519 entries, 0 to 44518
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      44519 non-null  object
 1   phishing_type  44519 non-null  object
 2   speaker        44519 non-null  int64 
 3   text           44519 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [ ]:
from konlpy.tag import Kkma

# 형태소 분석기 객체 생성
kkma = Kkma()


# 형태소 분석 함수 (불용어 제거 없이 주요 품사만 필터링)
def kkma_tokenizer(text):
    try:
        tokens = kkma.pos(text)
        # 명사(NNG, NNP), 동사(VV), 형용사(VA)만 추출
        keywords = [word for word, tag in tokens if tag in ["NNG", "NNP", "VV", "VA"]]
        keywords = [word for word in keywords if word != "OOO"]
        return keywords if keywords else ["없음"]
    except:
        return ["오류"]


# 적용
df["kkma_tokens"] = df["text"].apply(kkma_tokenizer)

# 결과 출력
# print(df)

In [ ]:
from collections import Counter
import pandas as pd

# 모든 키워드를 하나의 리스트로 합치기
all_keywords = sum(df["kkma_tokens"], [])  # 리스트 안 리스트를 평탄화

# 키워드 등장 빈도수 계산
keyword_counter = Counter(all_keywords)

# Counter → DataFrame 변환
keyword_freq_df = pd.DataFrame(
    keyword_counter.items(), columns=["keyword", "frequency"]
)

# 내림차순 정렬
keyword_freq_df = keyword_freq_df.sort_values(
    by="frequency", ascending=False
).reset_index(drop=True)

# 출력
# print(keyword_freq_df)
# 상위 700개 키워드만 추출
top_700_keywords = keyword_freq_df.head(700)

# 결과 확인
print(top_700_keywords)

In [ ]:
# top_700_keywords를 CSV 파일로 저장
top_700_keywords.to_csv("phishing_top_700_keywords.csv", index=False, encoding="utf-8-sig")